## Preprocessing

In [209]:
# Set up dependencies   
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import LambdaCallback
import tensorflow as tf
import pandas as pd 

In [210]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [211]:
# Drop the 'EIN' and 'NAME' columns.
colums_to_drop = ["EIN", "NAME"]    
application_df = application_df.drop(colums_to_drop, axis=1)
application_df.head()   

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [212]:
# Determine the number of unique values in each column.
application_df.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [213]:
# Look at APPLICATION_TYPE value counts for binning
application_type = application_df.APPLICATION_TYPE.value_counts()
print(application_type)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [214]:
# Cut off at 250 and create a list of application types to be replaced
cutoff_value_app = 250
application_types_to_replace = application_type[application_type < cutoff_value_app].index.tolist()

# Replace with "Other" in the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [215]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [216]:
# Look at CLASSIFICATION counts >1
classification_counts_upper = classification_counts[classification_counts > 1]
classification_counts_upper

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [217]:
# Cut off at 1000 and create a list of classifications to be replaced
cut_off_value_class = 1000  
classifications_to_replace = classification_counts[classification_counts < cut_off_value_class].index.tolist()  

# Replace with "Other" in the dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()  

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [218]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)

# Display the resulting DataFrame
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [219]:
# Split the preprocessed data into target and features arrays   
y = application_df_dummies["IS_SUCCESSFUL"].values
X = application_df_dummies.drop(["IS_SUCCESSFUL"], axis=1).values   

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [220]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [221]:
# Check the number of features in the training dataset  
number_input_features = len(X_train[0])
print(number_input_features)

43


In [224]:
# Define the model and architecture 
nn = tf.keras.models.Sequential()

# First hidden layer  
nn.add(tf.keras.layers.Dense(units=80, input_dim=number_input_features, activation='relu'))

# Second hidden layer 
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer 
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            (None, 80)                3520      
                                                                 
 dense_66 (Dense)            (None, 30)                2430      
                                                                 
 dense_67 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [225]:
# Create a lambda function to print callback after every 5 epochs   
print_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: print(f"Epoch {epoch + 1} - Loss: {logs['loss']}, Accuracy: {logs['accuracy']}") if (epoch + 1) % 5 == 0 else None
)

In [226]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [227]:
# Train the model with the callback
nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[print_callback])

Epoch 1/100
804/804 [==============================] - 1s 509us/step - loss: 0.5726 - accuracy: 0.7179
Epoch 2/100
804/804 [==============================] - 0s 498us/step - loss: 0.5551 - accuracy: 0.7319
Epoch 3/100
804/804 [==============================] - 0s 490us/step - loss: 0.5512 - accuracy: 0.7332
Epoch 4/100
804/804 [==============================] - 0s 487us/step - loss: 0.5501 - accuracy: 0.7319
Epoch 5/100
804/804 [==============================] - 0s 543us/step - loss: 0.5494 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 0s 480us/step - loss: 0.5472 - accuracy: 0.7353
Epoch 7/100
804/804 [==============================] - 0s 513us/step - loss: 0.5472 - accuracy: 0.7337
Epoch 8/100
804/804 [==============================] - 0s 568us/step - loss: 0.5467 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 0s 593us/step - loss: 0.5462 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 0s 599us/step - l

In [228]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5639 - accuracy: 0.7252 - 141ms/epoch - 528us/step
Loss: 0.5639389157295227, Accuracy: 0.7252478003501892


In [230]:
import warnings
# Suppress UserWarning (Native Keras is recommended, but we were instructed to save the model in HDF5 format) 
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

# Export the model to HDF5 file 
nn.save("AlphabetSoupCharity.h5")